In [137]:
# constants
from scipy import constants
import numpy as np

##### Constants (Peacemaker)

In [138]:
pi = 4.0 * np.arctan(1.0)
planck = 6.62606957e-34         # J s
avogadro = 6.0221413e23
kb = 1.3806488e-23              # J K^-1
speed_of_light = 299792458.0    # m s^-1
amu = 1.660538921e-27           # kg
gas_constant = avogadro*kb
hbar = planck/(2.0*pi)
global_eps = 1.0e-10

# Partition Functions

### Vibrational Partition Fuction (calculate_lnqvib)

#### Harmonic oscillator
$q_{\text{vib}} = \frac{\exp(-\Theta^{\text{vib}}/2T)}{1-\exp(-\Theta^{\text{vib}}/T)}$ </p>
$\Theta^{\text{vib}} = \frac{h\nu}{k_B}$ </p>
$\frac{h\nu}{k_BT} = \frac{hc}{k_BT} \cdot \~{v}$ 

In [139]:
# Case 1: 
#           Free rotator = False
#           Anharmonicity = 0

def lnqvib_case1(lnqvib, temp, freqs):
    """
    Calculate the vibrational partition function for a free rotator
    """
    # hc/k
    factor = planck * speed_of_light * 100 / kb 
    
    for i in range(len(freqs)):
        # Harmonic oscillator
        t_vib = factor * freqs[i]
        lnq = -t_vib / (2.0 * temp) - np.log(1 - np.exp(-t_vib / temp))
        
        lnqvib += lnq
    return lnqvib

#### Morse oscillator (Anharmonic effects in the quantum cluster equilibrium method -2017) 
$E_n = hc\nu [(n+ \frac{1}{2})-\mathcal{X}_e(n+ \frac{1}{2})^2]~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ 
$\mathcal{X}_e$ : the anharmonicity constant </p>
$q_i^{\text{morse}} = \prod_{\mu=1}^{\mu_{\text{max}}} \sum_{n=0}^{n^{\text{max}}} \exp(-(n+ \frac{1}{2})) \frac{\Theta_{i,\mu}^{\text{vib}}}{T}+\mathcal{X}_e(n+ \frac{1}{2})^2\frac{\Theta_{i,\mu}^{\text{vib}}}{T}~~~~~~$  $\mu$ : vibrational modes </p> 
$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ 
$n^{\text{max}}$ : highest vibrational energy level before dissociation for the respective mode </p>

$\Rightarrow$ Strekalov $\Rightarrow$ </p>
$q_i^{\text{morse}} = \prod_{\mu=1}^{\mu_{\text{max}}} \frac{1}{2 \cdot \sinh(\frac{\Theta_{i,\mu}^{\text{vib}}}{2T})} \exp(\mathcal{X}\frac{\Theta_{i,\mu}^{\text{vib}}}{T}(\frac{1}{4}+\frac{1}{2 \cdot \sinh(\frac{\Theta_{\mu}^{\text{vib}}}{2T})^2}))$ </p>

In [140]:
# Case 2:
#           Free rotator = False
#           Anharmonicity /= 0

def lnqvib_case2(lnqvib, temp, freqs, anharm_const):
    """
    Calculate the vibrational partition function for a free rotator
    """
    # hc/k
    factor = planck * speed_of_light * 100 / kb 
    
    for i in range(len(freqs)):
        # Morse oscillator
        t_vib = factor * freqs[i]
        # denk an die logaritmus gesetze
        lnqvib = lnqvib - np.log(2.0 * np.sinh(t_vib / (2.0 * temp))) + anharm_const * (t_vib / temp) * (0.25 + 1.0 / (2.0 * np.sinh(t_vib / (2.0 * temp))**2))
        
    return lnqvib

#### Free rotator (Supramolecular Binding Thermodynamics by Dispersion-Corrected Density Functional Theory - Grimme 2012)

$\mu = \frac{h}{8 \pi^2 \nu}$ </p>
$\mu' = \frac{\mu B_{\text{av}}}{\mu + B_{\text{av}}}~~~~~~~~~~~~~~~~~~~~~~~$ Moment of inertia of the cluster with frequency $\omega$</p>
$B_{\text{av}} = \sum_{n} \frac{I_n}{N}~~~~~~~~~~~~~~~~~~~$ Avarage moment of inertia of the cluster</p>
$T_{\text{rot}} = \frac{\hbar^2}{2 k_{\text{B}} \mu'}$ </p>
$q^{\text{rot}} = \frac{1}{\sigma} \sqrt{\frac{\pi T}{T_{\text{rot}}} }~~~~~~~~~~~~~~~~~$ $\sigma$ rotational symmetry nuber </p>
$w(\omega) = \frac{1}{1+(\text{rotor cutoff}/\nu)^4}$

In [141]:
# Case 3:
#           Free rotator = True
#           Anharmonicity = 0

def lnqvib_case3_4(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff):
    """
    Calculate the vibrational partition function for a free rotator
    """
    # hc/k
    factor = planck * speed_of_light * 100 / kb 
    Bav = sum(inertia)/len(inertia) * amu * 1.0e-20

    lnq_aho = 0.0
    lnq_ho = 0.0
    lnq_fr = 0.0
    lnqvib = 0.0
    w = 1.0
    
    for i in range(len(freqs)):
        if (rotor_cutoff > 0.0):
            mu = planck/(8.0 * pi**2 * freqs[i] * speed_of_light * 100.0)
            mu_prime = mu*Bav/(mu+Bav)
            trot = hbar**2/(2.0*kb*mu_prime)
            lnq_fr = np.log(1.0/sigma * (np.sqrt(pi*temp/trot)))
            w = 1.0/(1+(rotor_cutoff/freqs[i])**4)

        # Harmonic oscillator
        t_vib = factor * freqs[i]
        lnq_ho = -t_vib / (2.0 * temp) - np.log(1 - np.exp(-t_vib / temp))

        lnqvib += lnq_aho + w * lnq_ho + (1.0 - w) * lnq_fr
        
    return lnqvib

##### Examples

In [142]:
# Case 1: Example
temp = 298.00
freqs = [100, 200, 300]
lnqvib = 0.0#
lnqvib = lnqvib_case1(lnqvib, temp, freqs)
print('result',lnqvib)

result 0.2584469613283448


In [143]:
# Case 2: Example
temp = 298.00
freqs = [5.0, 648.0, 1000.0, 3555.7]
lnqvib = 0.0
anharm_const = 0.34
lnqvib = lnqvib_case2(lnqvib, temp, freqs, anharm_const)
print('result',lnqvib)

result 21.647769951871787


In [145]:
# Case 2: Example
temp = 298.00
freqs = [180.0, 270.0, 3550.0]

lnqvib = 0.0
anharm_const = 0.6
lnqvib = lnqvib_case2(lnqvib, temp, freqs, anharm_const)
print('result',lnqvib)

result -3.8012813311299443


In [144]:
# Case 3: Example
temp = 298.00
freqs = [100.0, 200.0, 300.0]
lnqvib = 0.0
inertia = [1.0, 1.0, 1.0]
sigma = 1
rotor_cutoff = 3000.0
lnqvib = lnqvib_case3_4(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff)
print('result',lnqvib)

# Case 3: Example
temp = 298.00
freqs = [5.0, 648.0, 1000.0, 3555.7]
lnqvib = 0.0
inertia = [3.2, 5.9, 89.3, 1.0]
sigma = 1
rotor_cutoff = 3000.0
lnqvib = lnqvib_case3_4(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff)
print('result',lnqvib)

result 1.7676272441463634
result -3.855446885285766
